In [14]:
# IMPORT ALL THE LIBRARIES!!!
import pandas as pd
import seaborn as sns
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

#Use Pandas Datareader
import pandas_datareader as web

import panel as pn
from panel.interact import interact
import plotly.express as px
pn.extension("plotly")

#Be certain to import hvplot AFTER panel extensions
import hvplot.pandas

%matplotlib inline

In [15]:
# Define variables for Yahoo Finance API call
# S&P500, DOW, NASDAQ, Russell2000

index_stocks = ['^GSPC', '^DJI', '^IXIC', '^RUT']
picked_stocks = ['FB', 'MSFT', 'GOOG', 'AAPL', 'COST']
random_stocks = ['CNFR', 'GAIA', 'FMS', 'RC']

start = pd.to_datetime('2015-10-01')
end = pd.to_datetime('2019-10-01')

fy19_start = pd.to_datetime('2019-10-01')
fy19_end = pd.to_datetime('2020-10-01')

combined_stocks = index_stocks + picked_stocks + random_stocks
combined_stocks.sort()


In [16]:
# Create function to check for CSV, then pull data into a dataframe, otherwise pull from yahoo finance and create a CSV and dataframe

stock_df = f"./csv/{combined_stocks[0]}-to-{combined_stocks[-1]}-from-{start.date()}-to-{end.date()}.csv"

if os.path.exists(stock_df): #if the csv file exists in directory, load file

    print(f"{stock_df} exists in this directory")

    stocks = pd.read_csv(stock_df, parse_dates=True, index_col="Date")

    fy19_stock_data = pd.read_csv('./csv/fy19_stock_data.csv', parse_dates=True, index_col="Date")

else: #otherwise pull fresh data

    print(f"Expected file does not exist, so a new file was created at {stock_df}.")

    stocks = web.DataReader(combined_stocks, 'yahoo', start, end)["Close"]

    # Grab fiscal year 2019 stock data
    fy19_stock_data = web.DataReader(combined_stocks, 'yahoo', fy19_start, fy19_end)["Close"]

    stocks.to_csv(f"./csv/{combined_stocks[0]}-to-{combined_stocks[-1]}-from-{start.date()}-to-{end.date()}.csv")

    fy19_stock_data.to_csv('./csv/fy19_stock_data.csv')


./csv/AAPL-to-^RUT-from-2015-10-01-to-2019-10-01.csv exists in this directory


In [17]:
# Create two plots to compare performance of stocks vs indexes

all_stocks_plot = stocks[(picked_stocks + random_stocks)].hvplot(title='Selected Stocks Price in USD')
index_stocks_plot = stocks[index_stocks].hvplot(title='Index Stocks Price in USD')

In [18]:
# Wrap those plots in a Panel Row

index_stocks_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [19]:
all_stocks_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [20]:
#Create new dataframe with the daily percent change of stocks

stocks_daily_return = stocks.pct_change().dropna()

fy19_daily_chg = fy19_stock_data.pct_change().dropna()

In [21]:
# Create function to plot scatter diagram of a stock against an index

def stocks_comparison(df, stock_ticker, title, index_ticker, color):
    return df.hvplot(title=title, kind='scatter', x=index_ticker, y=stock_ticker, color=color) 

# Create a beta line graph from index and stock tickers

def stock_beta_line(df, stock_ticker, index_ticker, color):
    b, a = np.polyfit(df[index_ticker], df[stock_ticker], 1)
    df_2 = pd.DataFrame(df[index_ticker], index=df.index)
    # df['stock'] = stocks_daily_return[index_ticker]
    df_2['Beta'] = b * df[index_ticker] + a
    # df = pd.DataFrame({'SPY': [stocks_daily_return[index_ticker]],  'GOOG': [(b * stocks_daily_return[index_ticker] + a)] })
    # df.hvplot('-', color = 'r')
    return df_2.hvplot(title=f"{b}", kind='line', x=index_ticker, y='Beta', color=color)


In [22]:
# Run Function on GOOG and all four indexes

def index_compare(df, stock_ticker, title, scatter_color, line_color):
    sp500_compare = stocks_comparison(df, stock_ticker, title, '^GSPC', scatter_color) * stock_beta_line(df, stock_ticker, '^GSPC', line_color)
    dow30_compare = stocks_comparison(df, stock_ticker, title, '^DJI', scatter_color) * stock_beta_line(df, stock_ticker, '^DJI', line_color)
    nasdaq_compare = stocks_comparison(df, stock_ticker, title, '^IXIC', scatter_color) * stock_beta_line(df, stock_ticker, '^IXIC', line_color)
    russell2000_compare = stocks_comparison(df, stock_ticker, title, '^RUT', scatter_color) * stock_beta_line(df, stock_ticker, '^RUT', line_color)
    
    return pn.Column(title, sp500_compare, dow30_compare, nasdaq_compare, russell2000_compare)

In [23]:
all_vs_index = pn.Row(index_stocks_plot, all_stocks_plot)
goog_comp = pn.Row(index_compare(stocks_daily_return, 'GOOG', "Google 2015-2019", 'orange', 'purple'), index_compare(fy19_daily_chg, 'GOOG', "Google 2020", 'green', 'red'))
fb_comp = pn.Row(index_compare(stocks_daily_return, 'FB', "Facebook 2015-2019", 'orange', 'purple'), index_compare(fy19_daily_chg, 'FB', "Facebook 2020", 'green', 'red'))
aapl_comp = pn.Row(index_compare(stocks_daily_return, 'AAPL', "Apple 2015-2019", 'orange', 'purple'), index_compare(fy19_daily_chg, 'AAPL', 'Apple 2020', 'green', 'red'))

In [24]:
# Create dashboard

alpha_beta_capm = pn.Tabs(
    ("Compare Stock Prices", all_vs_index),
    ("Compare Stock Beta: Google", goog_comp),
    ("Compare Stock Beta: Facebook", fb_comp),
    ("Compare Stock Beta: Apple", aapl_comp)
)

In [25]:
alpha_beta_capm.servable()

Tabs
    [0] Row
        [0] HoloViews(NdOverlay)
        [1] HoloViews(NdOverlay)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] HoloViews(Overlay)
            [2] HoloViews(Overlay)
            [3] HoloViews(Overlay)
            [4] HoloViews(Overlay)
        [1] Column
            [0] Markdown(str)
            [1] HoloViews(Overlay)
            [2] HoloViews(Overlay)
            [3] HoloViews(Overlay)
            [4] HoloViews(Overlay)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] HoloViews(Overlay)
            [2] HoloViews(Overlay)
            [3] HoloViews(Overlay)
            [4] HoloViews(Overlay)
        [1] Column
            [0] Markdown(str)
            [1] HoloViews(Overlay)
            [2] HoloViews(Overlay)
            [3] HoloViews(Overlay)
            [4] HoloViews(Overlay)
    [3] Row
        [0] Column
            [0] Markdown(str)
            [1] HoloViews(Overlay)
            [2] HoloViews(Overlay)
            [3] HoloViews(Overlay)
            [4] HoloViews(Overlay)
        [1] Column
            [0] Markdown(str)
            [1] HoloViews(Overlay)
            [2] HoloViews(Overlay)
            [3] HoloViews(Overlay)
            [4] HoloViews(Overlay)